In [2]:
# change working directory to the project root
import os
os.chdir('../../')

import sys
sys.path.append('models/utils')
sys.path.append('models/aln')

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import fitparams as fp

In [8]:
params = []
params = fp.loadpoint(params, "A2")

In [13]:
import aEIF_extended as IF_Models
EIFNeuron = IF_Models.EIFModel()

# DEFAULT VALUES
# check if BS parameters C and gL match with the AdEx parameters
print("aEIF C: {}".format(params['C']*1e-12))
print("aEIF gL: {}".format(params['gL']*1e-9))

#EIFNeuron.C_soma     = 1.0e-2  # Soma membrane capacitance (F / m2)
#EIFNeuron.rhom_soma  = 28.0e-1  # Soma membrane resistivity (Ohm m2)
#EIFNeuron.d_soma     = 10e-6   # Soma diameter ( m )

EIFNeuron.d_dend      = 2.0e-6  # Dendritic tree diameter (m)
EIFNeuron.L           = 1200.0e-6  # Dendritic tree length (m)

EIFNeuron.updateParams()

print("BS C: {}".format(EIFNeuron.C_s))
print("BS gL: {}".format(EIFNeuron.gL))

def current_for_adex(freq, EIFNeuron):
    # impedance
    z_adex = params['gL']*1e-9 * (1-1 * np.exp((-65-params['VT'])/params['DeltaT'])) + params['C']*1e-12 * 1j* 2*np.pi * freq
    z_bs = EIFNeuron.gL * (1 - 1 * np.exp((-65e-3-EIFNeuron.VT)/EIFNeuron.deltaT)) +  EIFNeuron.C_s * 1j* 2*np.pi * freq

    current = z_adex * EIFNeuron.polarizationTransfer(freq)
    return np.abs(current)

freq = 0

equivalent_current_for_1vm_dc_field = current_for_adex(freq, EIFNeuron)

print("Equivalent input current for a 1 V/m DC field (0 Hz) {} pA".format(equivalent_current_for_1vm_dc_field*1e12))

aEIF C: 2e-10
aEIF gL: 1e-08
BS C: 3.14159265359e-12
BS gL: 1.12199737628e-10
5.0239554247 pA


In [14]:
def adex(inp=0):
    I_syn = 0 # synaptic input, here 0
    C = 200
    gl = 10
    taum = C / gl
    deltaT = 1.5
    EL = -65
    EA = -80
    VT = -50
    Vr = -70
    dt = 0.1
    a = 0
    
    def I_exp(V):
        return gl * (EL - V) + gl * deltaT * np.exp((V - VT) / deltaT)

    V = Vr
    Ia = 0
    V_t = []
    Ia_t = []
    for i in range(10000):
        V_rhs = (I_exp(V) + I_syn + inp - Ia) / C
        Ia_rhs = a * (V - EA) - Ia
        
        V += V_rhs * dt
        V_t.append(V)
        
        Ia += Ia_rhs * dt
        Ia_t.append(Ia)
    return V_t

In [15]:
V_t_no_field = adex()
print("Resting membrane potential: {} mV".format(V_t_no_field[-1]))

V_t_field = adex(equivalent_current_for_1vm_dc_field) 
print("Resting membrane potential with 1 V/m DC field: {} mV".format(V_t_field[-1]))

Resting membrane potential: -64.999931897 mV
Resting membrane potential with 1 V/m DC field: -64.999931897 mV


In [169]:
print("{} mV increase from rest for 1 V/m".format(V_t_field[-1] - V_t_no_field[-1]))

0.502444635927 mV increase from rest for 1 V/m
